In [1]:
import pandas as pd 
import numpy as np 
import sklearn
import matplotlib.pyplot as plt

In [4]:
#Load preprocessed train and test 
train=pd.read_csv('../data/train.csv')
test=pd.read_csv('../data/test.csv')

In [ ]:
#MOTIVATION: We are not loan's domain experts, thus ... (see notes on ipad)

In [5]:
#### 4. Merging ####
#define a function to left join two datasets by handling separately numerical features and categorical ones
def join_w_stats(id, df1, df2, df2_name):
    """ Merge two dataframes (df1 and df2) by grouping df2 on id and computing the following statistics:
            i) Mean, Min and Max and sum for numeric features
            ii) Mean for categorical features 
        In this way, indeed, we hope to preserve the essence of the information stored in each feature after groub by"""

    #drop from df2 the id column since it is not necessary and won't be used anymore
    df2 = df2.drop([col for col in df2.columns if col.startswith('SK_ID') and col != id], axis=1)
    newcolumns = []
    
    
    #compute statistics for numerical feats, if there's any
    numericaldf2 = df2.select_dtypes(include='number')
    count_numericalcols = len(numericaldf2.columns)
    if count_numericalcols > 1:
        
        numericaldf2[id] = df2[id]
        numstatsdf2 = numericaldf2.groupby(id).agg(['mean', 'max', 'min', 'sum']).reset_index()

        #create new columns names for each numerical feature_stat
        for col in numstatsdf2.columns.levels[0]: 
            if col != id:
                #loop through every subcolumn name
                for stat in numstatsdf2.columns.levels[1][:-1]:
                    newcolumns.append('%s_%s_%s' % (df2_name, col, stat))

   
    #compute mean for categorical feats, if there's any
    categorical = False
    if (len(df2.columns) - count_numericalcols) > 0:
        categoricaldf2 = df2.select_dtypes(include='category')
        categorical = True
        onehotdf2 = onehot_binenc(categoricaldf2)
        onehotdf2[id] = df2[id]
        onehotstatsdf2 = onehotdf2.groupby(id).agg(['mean']).reset_index()
    
        #create new columns names for each categorical feature_stat
        for col in onehotstatsdf2.columns.levels[0]: 
            if col != id:
                #for categoricals the only subcolumn is the mean
                newcolumns.append('%s_%s_mean' % (df2_name, col))


    #merge both numerical and categorical (if there is any) statistics dsets grouped by id. And then with df1 
    if isinstance(df2, pd.DataFrame): 
        
        if categorical == True:
            numstatsdf2 = numstatsdf2.join(onehotstatsdf2.set_index(id), on=id)
            
        #add new columns names    
        numstatsdf2.columns = [id]+newcolumns 

        #left join on id df1 w/ merged statistics of df2
        df1joindf2 = df1.join(numstatsdf2.set_index(id), on=id)


    return df1joindf2

In [6]:
#Shrink down as much as we can the size of the dataframes.
#Note that every numerical value lies within the range indexed with a float/int of 32 bits
#Moreover is wise to convert every object feature into a category one, especially if the number of unique values is far from the number of rows

import sys

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [7]:
previousApplication = pd.read_csv('../data/previous_application.csv')
installmentsPayments = pd.read_csv('../data/installments_payments.csv')

In [8]:
previousApplication = convert_types(previousApplication, print_info=True)

Original Memory Usage: 0.49 gb.
New Memory Usage: 0.16 gb.


In [9]:
installmentsPayments = convert_types(installmentsPayments, print_info=True)

Original Memory Usage: 0.87 gb.
New Memory Usage: 0.44 gb.


In [10]:
previousJOINcashBalance = join_w_stats('SK_ID_PREV', previousApplication, installmentsPayments, 'installments')

In [11]:
import gc
# Remove variables to free memory
gc.enable()
del previousApplication, installmentsPayments 
gc.collect()

20

In [12]:
cashBalance = pd.read_csv('../data/POS_CASH_balance.csv')

In [13]:
cashBalance = convert_types(cashBalance, print_info=True)

Original Memory Usage: 0.64 gb.
New Memory Usage: 0.29 gb.


In [ ]:
cashBalance.head()

In [14]:
previousJOINcashBalanceJOINinstallments = join_w_stats('SK_ID_PREV', previousJOINcashBalance, cashBalance, 'cash')

/home/arbiterelegantiae/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ciao


In [15]:
import gc
# Remove variables to free memory
gc.enable()
del previousJOINcashBalance, cashBalance
gc.collect()

20

In [16]:
creditCardBalance = pd.read_csv('../data/credit_card_balance.csv')

In [17]:
creditCardBalance = convert_types(creditCardBalance, print_info=True)

Original Memory Usage: 0.71 gb.
New Memory Usage: 0.34 gb.


In [18]:
previousJOINcashBalanceJOINinstallmentsJOINcreditCardBalance = join_w_stats('SK_ID_PREV', previousJOINcashBalanceJOINinstallments, creditCardBalance, 'creditcard')

/home/arbiterelegantiae/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ciao


In [19]:
gc.enable()
del previousJOINcashBalanceJOINinstallments, creditCardBalance
gc.collect()

20

In [ ]:
previousJOINcashBalanceJOINinstallmentsJOINcreditCardBalance.shape

In [20]:
trainJOINprev = join_w_stats('SK_ID_CURR', train, previousJOINcashBalanceJOINinstallmentsJOINcreditCardBalance, 'prev')

/home/arbiterelegantiae/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ciao


In [23]:
gc.enable()
del previousJOINcashBalanceJOINinstallmentsJOINcreditCardBalance, train
gc.collect()

4519

In [24]:
trainJOINprev.shape

(307511, 917)

In [27]:
# remove all features with more than 70% of N.a.N
def remove_missing_columns(df, threshold = 60):
    # Calculate missing stats for df (remember to calculate a percent!)
    df_miss = pd.DataFrame(df.isnull().sum())
    df_miss['percent'] = 100 * df_miss[0] / len(df)
    
    
    # list of missing columns for df
    missing_df_columns = list(df_miss.index[df_miss['percent'] > threshold])
    
    # Print information
    print('There are %d columns with greater than %d%% missing values.' % (len(missing_df_columns), threshold))
    
    # Drop the missing columns and return
    df = df.drop(columns = missing_df_columns)
    
    return df

In [28]:
#drop all the new computed features that we consider no-influent from trainJOINprev
remove_missing_columns(trainJOINprev)


There are 267 columns with greater than 60% missing values.


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,prev_PRODUCT_COMBINATION_Cash X-Sell: low_mean,prev_PRODUCT_COMBINATION_Cash X-Sell: middle_mean,prev_PRODUCT_COMBINATION_POS household with interest_mean,prev_PRODUCT_COMBINATION_POS household without interest_mean,prev_PRODUCT_COMBINATION_POS industry with interest_mean,prev_PRODUCT_COMBINATION_POS industry without interest_mean,prev_PRODUCT_COMBINATION_POS mobile with interest_mean,prev_PRODUCT_COMBINATION_POS mobile without interest_mean,prev_PRODUCT_COMBINATION_POS other with interest_mean,prev_PRODUCT_COMBINATION_POS others without interest_mean
0,100002.0,0.0,0.0,1.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,0.0,1.0,0.0
1,100003.0,0.0,0.0,0.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,...,0.333333,0.00,0.333333,0.0,0.333333,0.0,0.000000,0.0,0.0,0.0
2,100004.0,1.0,1.0,1.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,0.000000,1.0,0.0,0.0
3,100006.0,0.0,0.0,1.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,...,0.222222,0.00,0.111111,0.0,0.111111,0.0,0.000000,0.0,0.0,0.0
4,100007.0,0.0,0.0,1.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,...,0.000000,0.50,0.166667,0.0,0.000000,0.0,0.166667,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251.0,0.0,0.0,0.0,254700.0,27558.0,225000.0,0.032561,-9327.0,-236.0,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0
307507,456252.0,0.0,0.0,1.0,269550.0,12001.5,225000.0,0.025164,-20775.0,-1648.0,...,0.000000,0.00,0.000000,1.0,0.000000,0.0,0.000000,0.0,0.0,0.0
307508,456253.0,0.0,0.0,1.0,677664.0,29979.0,585000.0,0.005002,-14966.0,-7921.0,...,0.000000,0.00,0.000000,0.0,0.000000,0.0,1.000000,0.0,0.0,0.0
307509,456254.0,0.0,0.0,1.0,370107.0,20205.0,319500.0,0.005313,-11961.0,-4786.0,...,0.000000,0.00,0.500000,0.0,0.000000,0.0,0.500000,0.0,0.0,0.0


In [30]:
trainJOINprev.to_csv('../data/trainjoinprev.csv', index=False)

In [29]:
bureauBalance = pd.read_csv('../data/bureau_balance.csv')
bureau = pd.read_csv('../data/bureau.csv')

In [31]:
bureauBalance = convert_types(bureauBalance, print_info=True)
bureau = convert_types(bureau, print_info=True)

Original Memory Usage: 0.66 gb.
New Memory Usage: 0.25 gb.
Original Memory Usage: 0.23 gb.
New Memory Usage: 0.1 gb.


In [32]:
bureauJOINbureauBalance = join_w_stats('SK_ID_BUREAU', bureau, bureauBalance, 'bureauBalance')

/home/arbiterelegantiae/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ciao


In [33]:
gc.enable()
del bureau, bureauBalance
gc.collect()

1608

In [36]:
trainJoined = join_w_stats('SK_ID_CURR', trainJOINprev, bureauJOINbureauBalance, 'bureau')

/home/arbiterelegantiae/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
ciao


In [37]:
gc.enable()
del trainJOINprev, bureauJOINbureauBalance
gc.collect()

4769

In [38]:
trainJoined.shape

(307511, 1036)

In [39]:
remove_missing_columns(trainJoined)

There are 306 columns with greater than 60% missing values.


,SK_ID_CURR,NAME_CONTRACT_TYPE,FLAG_OWN_CAR,FLAG_OWN_REALTY,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,...,bureau_CREDIT_TYPE_Interbank credit_mean,bureau_CREDIT_TYPE_Loan for business development_mean,bureau_CREDIT_TYPE_Loan for purchase of shares (margin lending)_mean,bureau_CREDIT_TYPE_Loan for the purchase of equipment_mean,bureau_CREDIT_TYPE_Loan for working capital replenishment_mean,bureau_CREDIT_TYPE_Microloan_mean,bureau_CREDIT_TYPE_Mobile operator loan_mean,bureau_CREDIT_TYPE_Mortgage_mean,bureau_CREDIT_TYPE_Real estate loan_mean,bureau_CREDIT_TYPE_Unknown type of loan_mean
0,100002.0,0.0,0.0,1.0,406597.5,24700.5,351000.0,0.018801,-9461.0,-637.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,100003.0,0.0,0.0,0.0,1293502.5,35698.5,1129500.0,0.003541,-16765.0,-1188.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004.0,1.0,1.0,1.0,135000.0,6750.0,135000.0,0.010032,-19046.0,-225.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006.0,0.0,0.0,1.0,312682.5,29686.5,297000.0,0.008019,-19005.0,-3039.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,100007.0,0.0,0.0,1.0,513000.0,21865.5,513000.0,0.028663,-19932.0,-3038.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251.0,0.0,0.0,0.0,254700.0,27558.0,225000.0,0.032561,-9327.0,-236.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307507,456252.0,0.0,0.0,1.0,269550.0,12001.5,225000.0,0.025164,-20775.0,-1648.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307508,456253.0,0.0,0.0,1.0,677664.0,29979.0,585000.0,0.005002,-14966.0,-7921.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
307509,456254.0,0.0,0.0,1.0,370107.0,20205.0,319500.0,0.005313,-11961.0,-4786.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# do the same mergings to for test as well and drop 60% empty columns 
# pca to reduce feature space? collinearity? correlations? noisy features removal? Tikhonov regularization 
# train model with boost library


In [ ]:
# Continue to the feat engineering phase